Group:

1) Last Name, First Name, Student Number:

2) Last Name, First Name, Student Number:


Goal: Learn how to manipulate 2D signals (=images) in both spectral and spatial domain.

**Due date**: 

Assignment due date is for Monday November 2nd, at 11h55 PM. A 3 points penalty per day will be applied in case of delay.

**Submission files** :

All code must be contained in the present template, as well as the answers to the questions (either commented within the code or with special Markdown/text cells).
Please follow the order of the subject.
Commenting the code is important and the overall clarity of the work will be taken in account. Make sure that every variable is clearly understandable and every figure readable.

You will also have to submit a static **HTML** version of this notebook *File->Download as...->HTML*. Put all your files (ipynb, html, eventually others externals ones) in a single **.zip** archive, named after your student number (StudentNb1_StudentNb2.zip).


# First session: Spatial filtering

**Note:** In your algorithms, take a special attention to the datatype of your arrays (*np.uint8*, *np.float32*, etc.). For image processing, it is probably going to be convenient to cast your arrays into  *np.float32*, then cast them back to *np.uint8* to display them.

**Note 2:** We highly recommand to use the *imshow* feature in the following way:
    
```python
    plt.imshow(mon_img, vmin=0, vmax=255)
    # ou bien
    ax.imshow(mon_img, vmin=0, vmax=255)
```
This forces matplotlib to interpretate values below or equal to 0 as black and values above or equal to 255 as white ([0, 255] being the possible value range in uint8). If you want to display images which values are between [0,1], you'll have to change *vmin* and *vmax* accordingly.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray' # Changing the default colormap, do not modify this.
import matplotlib
matplotlib.rcParams['figure.figsize'] = (15.0, 10.0) # Default figure size, you can modify this if you need to.

## Exercice 1 - Image Quality Enhancement
The goal of this exercise is to correct a heavily corrupted image. In order to do so, you will have to code a few of the filters you've seen in class.

### Question 1 (1.5 points)
Load the image *TempsModernes.jpeg* by using the function:
```python
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
```
(a) Display the image. As you can notice, it is very dark and has a very low contrast. Next to the image, plot the histogram of the image. You can use the code given in the next cell to do so. 

(b) We will fix the image's exposure and contrast by coding histogram equalization as seen in class.
Complete the following code:
```python
def equalize_histogram(img):
    imegale = np.zeros(img.shape)
    hist, bins = np.histogram(img, 256,[0,256])
    hist = hist.astype(np.float32)
    ...
```
Display the equalized image as well as its histogram. What do you observe?

In [2]:
def plot_histogram(img, ax=None):
    """
    param img: image as a np.ndarray, which values are between [0,255]
    param ax: (optional) matplotlib axe on which the histogram will be plotted. 
    If not provided, a new axe will be created.
    """
    hist, bins = np.histogram(img, 256,[0,256])
    if ax is not None:
        ax.bar(bins[:-1], hist)
    else:
        plt.bar(bins[:-1], hist)
        plt.show()

### Question 2 (1 point)
After the equalization process, what noise type seems predominant in the image? Explain why a median filter is well suited to remove it. Try it by filtering the image with a filter size of 5.


### Question 3 (1,5 points)
For the next step, you will have to use filtering by 2D convolution. Implement the function *conv2d* which takes an image and a mask as parameters and outputs the 2D convolution of both. You are only allowed to use matrix operators and maximum 2 *for* loops. You are not allowed to use any kind of already existing convolution function. But in order to maintain the same image size between the input and the output, you will have to do some **0-padding**. For that, you can take inspiration from the following code (to be completed). You can safely assume that the convolution mask is always going to be squared and its size odd.
```python
def conv2d(img, mask):
    out = np.zeros(img.shape, dtype=np.float32)
    size_mask = mask.shape[0]
    pad_values = ...
    img = np.pad(img, ((pad_values, pad_values), (pad_values, pad_values)))
    for i, row in enumerate(out):
        for j, col in enumerate(row):
            out[i, j] = ...
    return out

```

Try out your code by filtering your median-filtered image with the following gaussian mask:
```python
mask = np.asarray([[1,2,1,2,1],[2,4,8,4,2],[1,8,18,8,1],[2,4,8,4,2],[1,2,1,2,1]])/90
```
And display the result. Whad do you observe?

### Question 4 (1 point)
We will now fix the effect of the previous gaussian filter by implementing an edge-enhancement filter. To do so, we will use the  Laplacian of the image:
$\Delta(I)$. 
Implement your filter following the two following steps:
1. A low band filter gaussian filter is applied: $I_g = G * I$
2. $I_r = I_g + k\times \Delta(I_g)$ 
where $k$ is a constant you'll tune to obtain the best result.
As a reminder, the Laplacian can be easily obtained using a convolution with the following mask :
```python
mask_laplacian = np.asarray([[-1,-1,-1], [-1,8,-1], [-1,-1,-1]])
```
For the Gaussian filter, use the mask:
```python
mask_gaussian = np.asarray([[1,2,1], [2,4,2], [1,2,1]])
```


## Exercice 2 - Coin counter

You are now in charge of coding a coin counter. Since Canadian coins are mostly round, it's the perfect opportunity for you to test your knowledge in morphological filtering.

**Note**: There is an excellent [tutorial](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html) on morphological operators with *opencv-cv* that we highly recommend you to read.

### Question 1 (0,5 point)
Load the file **pieces.jpg** in grayscale. Display the image.


### Question 2 (0,5 point)
Implement the function
```python
def binarize(img, threshold):
    ...
```
that sets to 0 all pixels below a given threshold and to 255 all the others. This function takes an image and a threshold as parameters and outputs the thresholded image. Binarize your image with a threshold at 250. Of course, since we want to count the coins, they'll have to appear in white and you will have to invert the colors of your resulting binary image. Display it.


### Question 3 (1 point)
With the given threshold, small holes appears in your coins. Do a morphological closing in order to remove them, without modifying much the size of the coins. You will have to choose the approxiate structuring element. Display the result.

### Question 4 (3 points)
Implement the function:
```python
def count_coins(img_bin):
    ...
```
 which counts the total number of coins using erosion or dilation. The output of the function should be their sum in dollars.
 Here are some disk radius that might serve you : 200, 140, 120, 110, 90 (there are no coins of 1 dollar, 50 cents and 1 cent in the image). 

To count the number of connected components in an image, you can use the following function:
```python
def nb_components(img_bin):
    num_labels, labels_im = cv2.connectedComponents(img_bin)
    return num_labels-1 # We remove 1 because the background is considered as one connected component
```

# Séance 2: Fourier Transform and Spectral Filtering
## Exercise 3 2D FFT
The goal of this exercise is to familiarize yourself with the 2D-Fourier transform and some of its properties.

We are going to study the function:
$f(x, y) = \sin(\frac{2\pi}{256}[f_1 x+f_2 y] ) \text{  with  } x, y \in [0, 256]$

### Question 1 (1 point)
Create the function:
```python
def f(f1, f2):
    ...
```
that takes $\{f_1, f_2\}$ as parameters and outputs a monochromatic image of size 256x256. We highly suggest you to reead the documention of the function [np.meshgrid](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html) for an efficient implementation (without *for* loop).

### Question 2 (1,5 points)
For each of the following couple of parameters, display on adjancent axes the image $f(x, y)$ and its spectre (amplitude of the Fourier Transform). 

$\{f_1, f_2\}=$
1. \{12, 0\}
2. \{0, 12\}
2. \{12, 12\}
1. \{12, 32\}
1. \{32, -32\}

You'll have to do the computation of the spectre within a function:
```python
def fft_spectre(img):
    ...
```
that takes an image as a parameter and outputs the normalized spectre such that its maximal value is 1 and its minimal value is 0. Use the functions [np.fft.fft2](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft2.html#numpy.fft.fft2) and [np.fft.fftshift](https://numpy.org/doc/stable/reference/generated/numpy.fft.fftshift.html#numpy.fft.fftshift).
For the normalization, you can use the following relation:
$f_{normalisée} = \frac{f-\min(f)}{\max(f)-\min(f)}$
**Note:** Do not forget to add a title to all your figures to make them easily identifiable!

### Question 3 (1 point)
Complete the following sentences, using your observations from previous questions about the properties of the Fourier Transform.

In [3]:
prop = "The homothetic transformation (upscaling or downscaling) of an image along a given axis \
leads to %s of its spectre in Fourier domain."
ans = "..." 
print(prop%ans)


prop = "Rotation by an angle alpha of an image \
leads to %s of its spectre in Fourier domain."
ans = "..."
print(prop%ans)

The homothetic transformation (upscaling or downscaling) of an image along a given axis leads to ... of its spectre in Fourier domain.
Rotation by an angle alpha of an image leads to ... of its spectre in Fourier domain.


### Question 4 (1 point)
Given the mathematical function:

$f(x, y)=\sin(\frac{2\pi}{256}f_1r)$ où $r=\sqrt{x^2+y^2} \text{  avec  } x, y \in [-128, 128]$

Write the python fonction:
```python
def wave(f1):
    ...
```
that takes $f_1$ as a parameter and outputs an image of size 256x256.

As previsously, display on the same figure the pairs images/spectres for the following values of $f_1$:
$f_1=\{12,64,128,256\}$
What do you observe?

## Exercise 4  Spectral and homomorphic filtering

### Question 1 (1 point)
A mysterious file is provided, representing a famous portrait. Unfortunately, its identification is impossible because of a double parasite signal (sinusoidal), at the frequencies of $\pm 75$ along the each direction ($\pm x, \pm y$).



Load the file named **imageMystere.png**, displayed it as well as its Fourier Transform..

**Note:** Until now, we have only studied artificial images with a voluntarily spare spectral content. The spectre of the  mysterious image is way more dense, consequently its visualization is slightly more complicated (the many frequencies in the spectres have relatively small magnitude). To better distinguish what is going on, display the spectre in decibel:
```python
epsilon = 1e-7 # To avoid a log(0) error.
plt.imshow(20*np.log10(spectre+epsilon))
plt.show()
```
After a call to the fftshift function, the center of the spectre corresponds to the frequency (0,0). Verify you can see the signature of the parasites signals in the spectre.

### Question 2 (2 points)

We are going to eliminate the perturbations directly from the Fourier domain, by creating a filter that will be applied (through multiplication) to the fft.
The filter will have to be conceived according to one (or multiple) Gaussian profiles, following the formula:

$H(u, v) = 1-e^{-((u \pm f)^2+(v \pm f)^2)/\sigma }$ where $\sigma$ will allow to tune the selectivity of the filter and $f$ is a frequency to filter.


Adapt this formula in order to create a mask that will allow to get rid of the perturbations.
Display it on a figure. We will chose $\sigma=5$. What type of filter (low-band, high-band, ...) do you recognize? Visually, what does the parameter $\sigma$ ?

### Question 3 (1 point)

Use your mask to filter the Fourier Transform (**not just the magnitude!**) and reconstruct the image using the inverse Fourier Transform [np.fft.ifft2](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.fft.ifft2.html).
Display the reconstructed image and its spectre.

### Question 4 (1,5 points)
We recall that the definition of a homomorphic filter is $H(u, v)=(\gamma_H-\gamma_L)[1-e^{-c\frac{D^2(u, v)}{D^2_0}}]+\gamma_L$ where $D(u, v)=u^2+v^2$

Implement this filter and test it with the following parameters:
$D_0=2, \gamma_H=2, \gamma_L=0.5, c=1$.

Don't forget that the homomorphic filter is applied on the Fourier Transform of **logarithmic** image. Thus, once the filter is applied and back to the spatial domain, don't forget to take the **exponential** of the result.



Display the result (mask and filtered image) and describe the effect of the parameters $\gamma_H, \gamma_L$ and $D_0$.